In [26]:
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import os
import sys
sys.path.append(os.path.abspath('../src'))
import src.queries as q
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [2]:
franquicias_hist = pd.read_csv('../datos_finales/franquicias_historicas.csv')
standings_2003 = pd.read_csv('../datos_finales/standings_2003.csv')
salarios_jordan = pd.read_csv('../datos_finales/salarios_jordan.csv')
df_regular_season_jordan = pd.read_csv('../datos_finales/regular_season_jordan.csv')
df_temporadas_jordan = pd.read_csv('../datos_finales/temporadas_jordan.csv')
df_playoffs_jordan = pd.read_csv('../datos_finales/playoffs_jordan.csv')
df_campeonato_jordan = pd.read_csv('../datos_finales/campeonatos_jordan.csv')
standings_2024 = pd.read_csv('../datos_finales/standings_2024.csv')
salarios_lebron = pd.read_csv('../datos_finales/salarios_lebron.csv')
df_regular_season_lebron = pd.read_csv('../datos_finales/regular_season_lebron.csv')
df_playoff_lebron = pd.read_csv('../datos_finales/playoffs_lebron.csv')
df_temporadas_lebron = pd.read_csv('../datos_finales/temporadas_lebron.csv')
df_campeonato_lebron = pd.read_csv('../datos_finales/campeonatos_lebron.csv')

In [3]:
try:
    conexion = psycopg2.connect(
    database = 'GOAT',
    user = 'postgres',
    password = 'admin',
    host = 'localhost',
    port = '5432')
except OperationalError as e: #el alias es e
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print('la contraseña es erronea')
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print('error de conexion')
    else:
        print(f'el error es el siguiente {e}')

In [4]:
# # Cerrar el cursor si está abierto
# if cursor:
#     cursor.close()

# Cerrar la conexión si está abierta
# if conexion:
#     conexion.close()


In [5]:
cursor = conexion.cursor() #para crear mi primera tabla, independiente
tabla_campeonatos_mj = """
                   CREATE TABLE IF NOT EXISTS Campeonatos_MJ (
                   Id_resultado int PRIMARY KEY,
                   resultado VARCHAR(20)
                   );
                   """

cursor.execute(tabla_campeonatos_mj)
conexion.commit()

In [6]:
cursor = conexion.cursor()
tabla_campeonatos_lj = """
                   CREATE TABLE IF NOT EXISTS Campeonatos_LJ (
                   Id_resultado int PRIMARY KEY,
                   resultado VARCHAR(20)
                   );
                   """
cursor.execute(tabla_campeonatos_lj)
conexion.commit()

In [7]:
cursor = conexion.cursor()
franquicias = """
                   CREATE TABLE IF NOT EXISTS Franquicias (
                   Franquicia varchar (50),
                   team_code varchar(10) PRIMARY KEY,
                   liga varchar(10),
                   Desde varchar(10),
                   Hasta varchar(10),
                   Anios int,
                   G int,
                   W int, 
                   L int,
                   W_L_Perc float,
                   Plyfs int,
                   Div int,
                   Conf int,
                   Champ int
                   );
                   """
cursor.execute(franquicias)
conexion.commit()

In [8]:
cursor = conexion.cursor() 
tabla_lastdance_mj  = """
                                CREATE TABLE IF NOT EXISTS Tabla_2003 (
                                Equipo varchar(50),
                                Nombre_comun varchar(50),
                                team_code varchar(10),
                                Posicion int unique not null,
                                Partidos int,
                                Victorias int, 
                                Derrotas int,
                                PCTVIC float, 
                                CONSTRAINT fk_Franquicias FOREIGN KEY (team_code)
                                    REFERENCES Franquicias (team_code) 
                   );"""


cursor.execute(tabla_lastdance_mj)
conexion.commit()

In [9]:
cursor = conexion.cursor() 
tabla_lastdance_lj  = """
                                CREATE TABLE IF NOT EXISTS Tabla_2024 (
                                Equipo varchar(50),
                                Nombre_comun varchar(50),
                                team_code varchar(10),
                                Posicion int unique not null,
                                Partidos int,
                                Victorias int, 
                                Derrotas int,
                                PCTVIC float,
                                CONSTRAINT fk_Franquicias FOREIGN KEY (team_code)
                                    REFERENCES Franquicias (team_code) 
                   );"""

cursor.execute(tabla_lastdance_lj)
conexion.commit()

In [10]:
cursor = conexion.cursor()
Temp_mj = """
                   CREATE TABLE IF NOT EXISTS Temporadas_MJ (
                   Id_temporada int PRIMARY KEY,
                   Season varchar(10),
                   team_code varchar(10),
                   CONSTRAINT fk_Franquicias FOREIGN KEY (team_code)
                        REFERENCES Franquicias (team_code) 
                   );"""

cursor.execute(Temp_mj)
conexion.commit()

In [11]:
cursor = conexion.cursor()
Temp_lj = """
                   CREATE TABLE IF NOT EXISTS Temporadas_LJ (
                   Id_temporada int PRIMARY KEY,
                   Season varchar(10),
                   team_code varchar(10),
                   CONSTRAINT fk_Franquicias FOREIGN KEY (team_code)
                        REFERENCES Franquicias (team_code) 
                   );"""

cursor.execute(Temp_lj)
conexion.commit()

In [12]:
cursor = conexion.cursor()
temp_regular_mj = """
                   CREATE TABLE IF NOT EXISTS Temp_regular_MJ (
                   Id_temporada int,
                   team_code varchar(10),
                   player_age int,
                   GP int,
                   GS int,
                   Min float, 
                   FGM int,
                   FGA int,
                   FG_PCT float,
                   FG3M int,
                   FG3A int,
                   FG3_PCT float,
                   FTM int,
                   FTA int,
                   FT_PCT float,
                   OREB int,
                   DREB int,
                   REB int,
                   AST int,
                   STL int,
                   BLK int,
                   TOV int,
                   PF int, 
                   PTS int,
                   CONSTRAINT fk_Temporadas_MJ FOREIGN KEY (Id_temporada)
                        REFERENCES Temporadas_MJ (Id_temporada) 
                   );"""

cursor.execute(temp_regular_mj)
conexion.commit()

In [13]:
cursor = conexion.cursor()
temp_regular_lj = """
                   CREATE TABLE IF NOT EXISTS Temp_regular_LJ (
                   Id_temporada int,
                   team_code varchar(10),
                   player_age int,
                   GP int,
                   W int,
                   L int,
                   W_L_PCT float,
                   Min float, 
                   FGM int,
                   FGA int,
                   FG_PCT float,
                   FG3M int,
                   FG3A int,
                   FG3_PCT float,
                   FTM int,
                   FTA int,
                   FT_PCT float,
                   OREB int,
                   DREB int,
                   REB int,
                   AST int,
                   TOV int,
                   STL int,
                   BLK int,
                   BLKA int,
                   PF int,
                   PFD int, 
                   PTS int,
                   PLUS_MINUS int,
                   CONSTRAINT fk_Temporadas_LJ FOREIGN KEY (Id_temporada)
                        REFERENCES Temporadas_LJ (Id_temporada) 
                   );"""

cursor.execute(temp_regular_lj)
conexion.commit()

In [14]:
cursor = conexion.cursor()

playoffs_mj = """
                   CREATE TABLE IF NOT EXISTS Playoffs_MJ (
                   Id_temporada int,
                   player_age int,
                   team_code varchar(10),
                   G int,
                   GS int,
                   Min float, 
                   FGM float,
                   FGA float,
                   FG_PCT float,
                   FG3M float,
                   FG3A float,
                   FG3_PCT float,
                   FG2M float,
                   FG2A float,
                   FG2_PCT float, 
                   EFG_PCT float,
                   FTM float,
                   FTA float,
                   FT_PCT float,
                   OREB float,
                   DREB float,
                   REB float,
                   AST float,
                   STL float,
                   BLK float,
                   TOV float,
                   PF float, 
                   PTS float,
                   Id_Resultado int,
                   CONSTRAINT fk_Temporadas_MJ FOREIGN KEY (Id_temporada)
                        REFERENCES Temporadas_MJ (Id_temporada),
                   CONSTRAINT fk_Campeonatos_MJ FOREIGN KEY (Id_resultado)
                        REFERENCES Campeonatos_MJ (Id_resultado) 
                   );"""

cursor.execute(playoffs_mj)
conexion.commit()

In [15]:
cursor = conexion.cursor()
playoffs_lj = """
                   CREATE TABLE IF NOT EXISTS Playoffs_LJ (
                   Id_temporada int,
                   player_age int,
                   team_code varchar(10),
                   G int,
                   GS int,
                   Min float, 
                   FGM int,
                   FGA int,
                   FG_PCT float,
                   FG3M float,
                   FG3A float,
                   FG3_PCT float,
                   FG2M float,
                   FG2A float,
                   FG2_PCT float, 
                   EFG_PCT float,
                   FTM float,
                   FTA float,
                   FT_PCT float,
                   OREB float,
                   DREB float,
                   REB float,
                   AST float,
                   STL float,
                   BLK float,
                   TOV float,
                   PF float, 
                   PTS float,
                   Id_Resultado int,
                   CONSTRAINT fk_Temporadas_LJ FOREIGN KEY (Id_temporada)
                        REFERENCES Temporadas_LJ (Id_temporada),
                   CONSTRAINT fk_Campeonatos_LJ FOREIGN KEY (Id_resultado)
                        REFERENCES Campeonatos_LJ (Id_resultado) 
                   );"""

cursor.execute(playoffs_lj)
conexion.commit()

In [16]:
cursor = conexion.cursor()
salarios_mj = """
                   CREATE TABLE IF NOT EXISTS salarios_mj (
                   team_code varchar(10),
                   salario int,
                   salario_actual int,
                   Id_temporada int, 
                   CONSTRAINT fk_Temporadas_MJ FOREIGN KEY (Id_temporada)
                        REFERENCES Temporadas_MJ (Id_temporada)
                   );"""

cursor.execute(salarios_mj)
conexion.commit()

In [17]:
cursor = conexion.cursor()
salarios_lj = """
                   CREATE TABLE IF NOT EXISTS salarios_lj (
                   team_code varchar(10),
                   salario int,
                   salario_actual int,
                   Id_temporada int, 
                   CONSTRAINT fk_Temporadas_LJ FOREIGN KEY (Id_temporada)
                        REFERENCES Temporadas_LJ (Id_temporada)
                   );"""

cursor.execute(salarios_lj)
conexion.commit()

In [18]:
standings_2003.head()

,Equipo,Nombre_comun,Codigo_equipo,Posicion,Partidos,Victorias,Derrotas,Porcentaje_vic
0,Phoenix Suns,Suns,PHX,1,82,64,18,0.780
1,Memphis Grizzlies,Grizzlies,MEM,2,82,56,26,0.683
2,Golden State Warriors,Warriors,GSW,3,82,53,29,0.646
3,Miami Heat,Heat,MIA,4,82,53,29,0.646
4,Dallas Mavericks,Mavericks,DAL,5,82,52,30,0.634


Carga de datos

In [19]:
# crea una lista tuplas a partir de los valores de mis columnas
lista_tuplas = list(zip(df_campeonato_jordan['Id_resultado'], df_campeonato_jordan['Resultado']))

query_carga_1 = 'INSERT INTO Campeonatos_MJ (Id_resultado, resultado) VALUES (%s, %s)'

# Ejecutar la consulta con executemany para insertar todas las filas de una vez
cursor.executemany(query_carga_1, lista_tuplas)
conexion.commit()

In [20]:
# son tablas similares, repito el proceso anterior
lista_tuplas_1 = list(zip(df_campeonato_lebron['Id_resultado'], df_campeonato_lebron['Resultado']))

query_carga_2 = 'INSERT INTO Campeonatos_LJ (Id_resultado, resultado) VALUES (%s, %s)' 
cursor.executemany(query_carga_2, lista_tuplas_1)
conexion.commit() # inserto valores a mi tabla

In [21]:
lista_tuplas_2 = []
for fila in franquicias_hist.itertuples(index=False, name=None): #genera una lista de tuplas, donde omite el indice y evito que pase a objeto de Pandas
    lista_tuplas_2.append(fila)

query_carga_3 = """INSERT INTO Franquicias 
(Franquicia, team_code, liga, Desde, Hasta, Anios, G, 
W, L, W_L_Perc, Plyfs, Div, Conf, Champ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""" # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas_2:
    cursor.execute(query_carga_3, valores)
conexion.commit() # inserto valores a mi tabla

In [ ]:
# Generar lista de tuplas asegurando que cada tupla tenga el número correcto de columnas
lista_tuplas_3 = [tuple(fila) for fila in standings_2003.itertuples(index=False, name=None)]

query_carga_4 = """
    INSERT INTO Tabla_2003 
    (Equipo, Nombre_comun, team_code, Posicion, Partidos, Victorias, Derrotas, PCTVIC) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""  

for valores in lista_tuplas_3:
    cursor.execute(query_carga_4, valores)

conexion.commit()


In [23]:
lista_tuplas_4 = []
for fila in standings_2024.itertuples(index=False, name=None):
    lista_tuplas_4.append(tuple(fila))

query_carga_5 = """INSERT INTO Tabla_2024
(Equipo, Nombre_comun, team_code, Posicion, Partidos, Victorias, 
Derrotas, PCTVIC) VALUES (%s, %s,%s, %s, %s, %s, %s, %s)""" # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas_4:
    cursor.execute(query_carga_5, valores)
conexion.commit() # inserto valores a mi tabla

In [24]:
lista_tuplas_5 = []
for fila in df_temporadas_jordan.itertuples(index=False, name=None):
    lista_tuplas_5.append(tuple(fila))

query_carga_6 = 'INSERT INTO Temporadas_MJ (Id_temporada, Season, team_code) VALUES (%s, %s, %s)' 
for valores in lista_tuplas_5:
    cursor.execute(query_carga_6, valores)
conexion.commit() # inserto valores a mi tabla

In [25]:
lista_tuplas_6 = []
for fila in df_temporadas_lebron.itertuples(index=False, name=None):
    lista_tuplas_6.append(tuple(fila))

query_carga_7 = 'INSERT INTO Temporadas_LJ (Id_temporada, Season, team_code) VALUES (%s, %s, %s)' 
for valores in lista_tuplas_6:
    cursor.execute(query_carga_7, valores)
conexion.commit() # inserto valores a mi tabla

In [26]:
lista_tuplas_7 = []
for fila in df_regular_season_jordan.itertuples(index=False, name=None):
    lista_tuplas_7.append(tuple(fila))

query_carga_8 = """INSERT INTO Temp_regular_MJ (Id_temporada, team_code, player_age, GP, GS,
Min, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, 
FT_PCT, OREB, DREB, REB, AST, STL, BLK, TOV, PF, PTS) VALUES 
(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s ,%s ,%s, %s, %s, %s ,%s ,%s ,%s, %s ,%s, %s)""" # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas_7:
    cursor.execute(query_carga_8, valores)
conexion.commit() # inserto valores a mi tabla

In [27]:
lista_tuplas_8 = []
for fila in df_regular_season_lebron.itertuples(index=False, name=None):
    lista_tuplas_8.append(tuple(fila))

query_carga_9 = """INSERT INTO Temp_regular_LJ (Id_temporada, team_code, player_age, GP, W, L, W_L_PCT,
Min, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, 
FT_PCT, OREB, DREB, REB, AST, TOV, STL, BLK, BLKA, PF, PFD, PTS, PLUS_MINUS) VALUES 
(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s ,%s ,%s, %s, %s, %s ,%s ,%s ,%s, %s ,%s, %s, %s, %s, %s, %s, %s )""" # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas_8:
    cursor.execute(query_carga_9, valores)
conexion.commit() # inserto valores a mi tabla

In [28]:
lista_tuplas_9 = []
for fila in df_playoffs_jordan.itertuples(index=False, name=None):
    lista_tuplas_9.append(tuple(fila))

query_carga_10 = """INSERT INTO Playoffs_MJ (Id_temporada, player_age, team_code, G, GS,
Min, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FG2M, FG2A, FG2_PCT, EFG_PCT, FTM, FTA,
FT_PCT, OREB, DREB, REB, AST, STL, BLK, TOV, PF, PTS, Id_Resultado) VALUES 
(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s ,%s ,%s, %s, %s, %s ,%s ,%s ,%s, %s ,%s, %s, %s, %s, %s, %s, %s)""" # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas_9:
    cursor.execute(query_carga_10, valores)
conexion.commit() # inserto valores a mi tabla

In [ ]:
lista_tuplas_10 = []
for fila in df_playoff_lebron.itertuples(index=False, name=None):
    lista_tuplas_10.append(tuple(fila))

query_carga_11 = """INSERT INTO Playoffs_LJ (Id_temporada, player_age, team_code, G, GS,
Min, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FG2M, FG2A, FG2_PCT, EFG_PCT , FTM, FTA,
FT_PCT, OREB, DREB, REB, AST, STL, BLK, TOV, PF, PTS, Id_Resultado) VALUES 
(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s ,%s ,%s, %s, %s, %s ,%s ,%s ,%s, %s ,%s, %s, %s, %s, %s, %s, %s)""" # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas_10:
    cursor.execute(query_carga_11, valores)
conexion.commit() # inserto valores a mi tabla

In [34]:
lista_tuplas_11 = []
for fila in salarios_jordan.itertuples(index=False, name=None):
    lista_tuplas_11.append(tuple(fila))

query_carga_12 = """INSERT INTO salarios_mj (team_code, salario, salario_actual, Id_temporada) VALUES 
(%s, %s, %s, %s)""" # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas_11:
    cursor.execute(query_carga_12, valores)
conexion.commit() # inserto valores a mi tabla

In [35]:
lista_tuplas_12 = []
for fila in salarios_lebron.itertuples(index=False, name=None):
    lista_tuplas_12.append(tuple(fila))

query_carga_13 = """INSERT INTO salarios_lj (team_code, salario, salario_actual, Id_temporada) VALUES 
(%s, %s, %s, %s)""" # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas_12:
    cursor.execute(query_carga_13, valores)
conexion.commit() # inserto valores a mi tabla

In [24]:
# Cerrar el cursor si está abierto
if cursor:
    cursor.close()

# Cerrar la conexión si está abierta
if conexion:
    conexion.close()


* EDA

In [10]:
cursor=conexion.cursor()
query_EDA_1 = """
            SELECT id_temporada, player_age, team_code, fg_pct, min, pts
FROM campeonatos_lj cl
INNER JOIN playoffs_lj pl 
ON cl.id_resultado = pl.id_resultado 
WHERE pl.id_resultado = 1
"""
cursor.execute(query_EDA_1)
temp_camp_lj = pd.DataFrame(cursor.fetchall())

In [11]:
temp_camp_lj.head()

,0,1,2,3,4,5
0,9,27,MIA,0.500,42.7,30.3
1,10,28,MIA,0.491,41.7,25.9
2,13,31,CLE,0.525,39.1,26.3
3,17,35,LAL,0.560,36.3,27.6


In [12]:
cursor=conexion.cursor()
query_EDA_2 = """
SELECT id_temporada, player_age, team_code, fg_pct, min, pts
FROM campeonatos_mj cm 
INNER JOIN playoffs_mj pm 
ON cm.id_resultado = pm.id_resultado 
WHERE pm.id_resultado = 1;
"""
cursor.execute(query_EDA_2)
temp_camp_mj = pd.DataFrame(cursor.fetchall())

In [13]:
temp_camp_mj.head()

,0,1,2,3,4,5
0,7,27,CHI,0.524,40.5,31.1
1,8,28,CHI,0.499,41.8,34.5
2,9,29,CHI,0.475,41.2,35.1
3,11,32,CHI,0.459,40.7,30.7
4,12,33,CHI,0.456,42.3,31.1


In [14]:
cursor=conexion.cursor()
query_EDA_3 = """
SELECT sl.team_code, salario, salario_actual, sl.id_temporada, tl.season 
FROM salarios_lj sl 
INNER JOIN temporadas_lj tl 
ON sl.id_temporada = tl.id_temporada 
ORDER BY sl.id_temporada ASC ;
"""
cursor.execute(query_EDA_3)
sal_lj_temp = pd.DataFrame(cursor.fetchall())

In [15]:
sal_lj_temp.head()

,0,1,2,3,4
0,CLE,4018920,6675060,1,2003-04
1,CLE,4320360,6948764,2,2004-05
2,CLE,4621800,7250142,3,2005-06
3,CLE,5828090,8763936,4,2006-07
4,CLE,13041250,19097502,5,2007-08


In [16]:
cursor=conexion.cursor()
query_EDA_4 = """
SELECT sum(salario_actual), team_code, count(id_temporada) 
FROM salarios_lj sl 
GROUP BY team_code;
"""
cursor.execute(query_EDA_4)
sal_mj_acum = pd.DataFrame(cursor.fetchall())

In [17]:
sal_mj_acum.head()

,0,1,2
0,273828768,LAL,6
1,227603713,CLE,11
2,90196941,MIA,4


In [27]:
cursor=conexion.cursor()

cursor.execute(q.query_EDA_5)
sal_mj_acum = pd.DataFrame(cursor.fetchall())

AttributeError: module 'src.queries' has no attribute 'query_EDA_5'